# Onnx to Circom

## Parameters extraction

### Onnx model

In [9]:
import onnx
import onnxruntime

# Load the ONNX model
model = onnx.load('model.onnx')

# Create an inference session to compute the input size
session = onnxruntime.InferenceSession('model.onnx')

# Iterate over the nodes
num = 0
for node in model.graph.node:
    num+=1
    if node.op_type in useful_operations:
        break

NameError: name 'useful_operations' is not defined

### Tensorflow model

In [8]:
import tensorflow as tf

# Create a simple model with a Dense layer
model_tf = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=5, kernel_size=3, input_shape=(4, 10, 32)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=30, kernel_size=2, ), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    #tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.MaxPooling2D(pool_size=(1, 1)),
    #tf.keras.layers.GlobalMaxPool2D(),
    #tf.keras.layers.GlobalAveragePooling2D(),
    #tf.keras.layers.Flatten(name='layer_6'),
    tf.keras.layers.Dense(4, activation='softmax'),
])

In [158]:
model_tf.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 2, 8, 5)           1445      
                                                                 
 batch_normalization_18 (Bat  (None, 2, 8, 5)          20        
 chNormalization)                                                
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 7, 30)          630       
                                                                 
 batch_normalization_19 (Bat  (None, 1, 7, 30)         120       
 chNormalization)                                                
                                                                 
 dense_38 (Dense)            (None, 1, 7, 8)           248       
                                                                 
 dense_39 (Dense)            (None, 1, 7, 12)        

#### First input shape

In [141]:
# First input
for input in model.graph.input:
    print('Input tensor name: ', input.name)
    print('Input tensor shape: ', [dim.dim_value for dim in input.type.tensor_type.shape.dim])

Input tensor name:  input
Input tensor shape:  [0, 4, 10, 32]


In [208]:
node = model.graph.node[8]
layer = model_tf.layers[0]

### Layer Traslation

#### Conv layer

In [10]:
layer = model_tf.layers[0]
node = model.graph.node[8]

##### Layer name

In [147]:
layer.name, node.op_type

('conv2d_9', 'Conv')

##### Input
The input shape is the same of the previous layer's output shape or the first input calculated above

In [140]:
# It is the shape of the input tensor
layer.input

<KerasTensor: shape=(None, 4, 10, 32) dtype=float32 (created by layer 'conv2d_9_input')>

##### Weights (Shape and values)
To be given thanks to json file

In [160]:
layer.weights[0].shape

#ONNX: load from json to security reasons

TensorShape([3, 3, 32, 5])

##### Bias (bias) (Shapes and values)
To be given thanks to json file

In [183]:
layer.weights[1].shape

#ONNX: load from json to security reasons

TensorShape([5])

##### Layer output
To be calculated, in case of Conv layer:

- H_out = [(H_in - F_height + 2 * Padding) / S_height] + 1

- W_out = [(W_in - F_width + 2 * Padding) / S_width] + 1

- C_out = Number of filters in the convolutional layer

In [184]:
layer.output #input: 4, 10, 32

<KerasTensor: shape=(None, 2, 8, 5) dtype=float32 (created by layer 'conv2d_13')>

In [189]:
# (filter dim1, filter dim2, input channels, output channels)
layer.get_weights()[0].shape, layer.padding

Kernel shape:  [3, 3]


In [190]:
# filter dim1, filter dim2
for attribute in node.attribute:
    if attribute.name == 'kernel_shape':
        kernel_shape = attribute.ints[1]
        print('Kernel shape: ', [kernel_shape, kernel_shape])

Kernel shape:  [3, 3]


##### nRows

In [186]:
layer.input[0] # 4

<KerasTensor: shape=(4, 10, 32) dtype=float32 (created by layer 'tf.__operators__.getitem_18')>

##### nCol

In [146]:
layer.input[1] # 10

<KerasTensor: shape=(4, 10, 32) dtype=float32 (created by layer 'tf.__operators__.getitem_16')>

##### nChannels & nFilters

In [166]:
# The number of channels is egual to the number of filters, 
# and it's egual to the number of dimensione 3 of wheits array
layer.filters, layer.weights[0].shape[3]

(5, 5)

In [169]:
for node in model.graph.node:
    if node.op_type == 'Conv':  # We found a convolutional layer
        # Get the name of the weights tensor for this layer
        weights_name = node.input[1]  # The weights tensor is usually the second input
        # Now we look for this tensor in the initializers
        for initializer in model.graph.initializer:
            if initializer.name == weights_name:
                # We found the weights tensor. Its shape gives us the number of filters
                weights = numpy_helper.to_array(initializer)
                print('Layer (node) name: ', node.name)
                print('Number of filters: ', weights.shape[0])
                break

Layer (node) name:  sequential/conv1d/Conv1D/Conv2D
Number of filters:  5
Layer (node) name:  sequential/conv2d/Conv2D
Number of filters:  30


##### KernelSize

In [171]:
layer.kernel_size

(3, 3)

In [179]:
for attribute in node.attribute:
    if attribute.name == 'kernel_shape':
        kernel_shape = attribute.ints[1]
        print('Kernel shape: ', [kernel_shape, kernel_shape])

Kernel shape:  [3, 3]


##### Strides

In [182]:
for attribute in node.attribute:
    if attribute.name == 'strides':
        stride_shape = attribute.ints
        print('Strides shape: ', stride_shape)

Strides shape:  [1, 1]


#### Circuit Conv Creation

In [1]:
from keras2circom.keras2circom import circom
from keras2circom.keras2circom.circom import Component, Circuit, Signal, Template
import typing


In [2]:
import os
from os import path
import re

def dir_parse(dir_path, skips=[]):
    '''parse circom files in a directory'''
    names = os.listdir(dir_path)
    for name in names:
        if name in skips:
            continue

        fpath = path.join(dir_path, name)
        if os.path.isdir(fpath):
            dir_parse(fpath)
        elif os.path.isfile(fpath):
            if fpath.endswith('.circom'):
                templates = file_parse(fpath)
    print(templates)
    return templates

def file_parse(fpath):
    '''parse circom file and register templates'''
    with open(fpath, 'r') as f:
        lines = f.read().split('\n')

    lines = [l for l in lines if not l.strip().startswith('//')]
    lines = ' '.join(lines)

    lines = re.sub('/\*.*?\*/', 'IGN', lines)

    funcs = re.findall('template (\w+) ?\((.*?)\) ?\{(.*?)\}', lines)
    for func in funcs:
        op_name = func[0].strip()
        args = func[1].split(',')
        main = func[2].strip()
        assert op_name not in templates, \
            'duplicated template: {} in {} vs. {}'.format(
                    op_name, templates[op_name].fpath, fpath)

        signals = re.findall('signal (\w+) (\w+)(.*?);', main)
        infos = [[] for i in range(4)]
        for sig in signals:
            sig_types = ['input', 'output']
            assert sig[0] in sig_types, sig[1] + ' | ' + main
            idx = sig_types.index(sig[0])
            infos[idx*2+0].append(sig[1])

            sig_dim = sig[2].count('[')
            infos[idx*2+1].append(sig_dim)
        templates[op_name] = Template(
                op_name, fpath,
                [a.strip() for a in args],
                *infos)
    return templates

In [3]:
templates = {}
templates = dir_parse('node_modules/circomlib-ml/circuits/', skips=['util.circom', 'circomlib-matrix', 'circomlib', 'crypto'])

{'ArgMax': Template(op_name='ArgMax', fpath='node_modules/circomlib-ml/circuits/ArgMax.circom', args=['n'], input_names=['in'], input_dims=[1], output_names=['out'], output_dims=[0]), 'AveragePooling2D': Template(op_name='AveragePooling2D', fpath='node_modules/circomlib-ml/circuits/AveragePooling2D.circom', args=['nRows', 'nCols', 'nChannels', 'poolSize', 'strides', 'scaledInvPoolSize'], input_names=['in'], input_dims=[3], output_names=['out'], output_dims=[3]), 'BatchNormalization2D': Template(op_name='BatchNormalization2D', fpath='node_modules/circomlib-ml/circuits/BatchNormalization2D.circom', args=['nRows', 'nCols', 'nChannels'], input_names=['in', 'a', 'b'], input_dims=[3, 1, 1], output_names=['out'], output_dims=[3]), 'Conv1D': Template(op_name='Conv1D', fpath='node_modules/circomlib-ml/circuits/Conv1D.circom', args=['nInputs', 'nChannels', 'nFilters', 'kernelSize', 'strides'], input_names=['in', 'weights', 'bias'], input_dims=[2, 3, 1], output_names=['out'], output_dims=[2]), 'C

In [4]:
import numpy as np
array = (np.random.random(1458) * 255).astype(np.uint8)

In [5]:
print("[", end="")
for i in array.flatten():
    print(str(i)+ ",",  end=" ")
print("]")

[49, 191, 121, 157, 152, 181, 133, 47, 204, 128, 132, 85, 161, 16, 51, 18, 62, 105, 233, 208, 30, 183, 108, 143, 71, 144, 171, 39, 171, 103, 125, 39, 73, 117, 156, 161, 208, 161, 215, 62, 127, 230, 145, 244, 27, 63, 248, 106, 199, 10, 6, 208, 68, 138, 173, 86, 233, 177, 221, 109, 190, 143, 246, 186, 65, 180, 125, 108, 177, 217, 46, 144, 162, 57, 91, 121, 147, 29, 133, 59, 87, 158, 110, 6, 154, 222, 245, 97, 82, 70, 230, 108, 19, 165, 123, 139, 142, 80, 19, 222, 69, 155, 97, 128, 197, 203, 145, 207, 176, 159, 79, 79, 202, 15, 219, 128, 182, 22, 222, 127, 18, 181, 101, 159, 55, 161, 225, 192, 197, 180, 207, 207, 69, 65, 129, 74, 163, 4, 239, 83, 181, 208, 48, 104, 61, 99, 133, 141, 180, 237, 248, 57, 85, 180, 3, 100, 65, 186, 214, 228, 241, 7, 157, 191, 44, 213, 182, 229, 217, 58, 247, 69, 149, 62, 23, 167, 93, 25, 94, 22, 61, 6, 192, 56, 17, 54, 218, 226, 26, 236, 200, 108, 243, 32, 131, 233, 183, 222, 208, 88, 235, 85, 27, 59, 152, 139, 167, 103, 207, 94, 122, 68, 145, 1, 18, 13, 63, 2

In [28]:
layer.input

<KerasTensor: shape=(None, 4, 10, 32) dtype=float32 (created by layer 'conv2d_input')>

In [37]:
conv = Component(layer.name, templates['Conv2D'], [
    Signal('in', layer.input[1]),
    Signal('weights', layer.weights[0].shape, layer.weights[0]),
    Signal('bias', layer.weights[1].shape, layer.weights[1]),
    ],[Signal('out', layer.output)],{
    'nRows': layer.input[0],
    'nCols': layer.input[1],
    'nChannels': layer.input[2],
    'nFilters': layer.filters,
    'kernelSize': layer.kernel_size,
    'strides': layer.strides,
    })


In [36]:
layer.input

<KerasTensor: shape=(None, 4, 10, 32) dtype=float32 (created by layer 'conv2d_input')>

In [38]:
circuit = Circuit()
circuit.add_components([conv])

In [45]:
type(layer.output)

keras.engine.keras_tensor.KerasTensor

In [50]:
circuit.to_json()

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'round'

In [24]:
with open('./circuit_conv.json', 'w') as f:
    f.write(circuit.to_json())

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'round'

In [25]:
with open('./circuit_conv.circom', 'w') as f:
    f.write(circuit.to_circom())

TypeError: Cannot iterate over a Tensor with unknown first dimension.